<a href="https://colab.research.google.com/github/harshitharavi26/PP1/blob/main/Final_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import os
import cv2    
from tqdm import tqdm
import random as random
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.utils import to_categorical
from keras.layers import Dense,Dropout,Flatten,Conv2D,MaxPool2D,BatchNormalization
from keras.models import Sequential
from keras.losses import categorical_crossentropy
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator

import warnings
warnings.filterwarnings('always')
warnings.filterwarnings('ignore')

In [3]:
imagedetails = []
imagelabels = []
IMG_SIZE=150
diseased_dir='/content/drive/My Drive/diseased_chicken/Diseased'
healthy_dir = '/content/drive/My Drive/diseased_chicken/Healthy'

In [4]:
def read_image(label,DIR):
    
    for img in tqdm(os.listdir(DIR)):
    #for img in DIR:
        path = os.path.join(DIR,img)
        #checking image type
        _, ftype = os.path.splitext(path)
        image = cv2.imread(path,cv2.IMREAD_COLOR)
        image = cv2.resize(image, (IMG_SIZE,IMG_SIZE),interpolation = cv2.INTER_AREA)
        imagedetails.append(np.array(image))
        imagelabels.append(str(label))

read_image("Diseased",diseased_dir)
read_image("Healthy",healthy_dir)

100%|██████████| 99/99 [00:31<00:00,  3.13it/s]


In [5]:
encoder = LabelEncoder()
#converting the images into arrays which will have its intensity values
X = np.array(imagedetails)
#divide by 255 is to make the analysis easy because each color 8 plains (2^8==256)
X = X/255
#transformig the image labels into integers because it easy to deal with integers compared to strings;
y = encoder.fit_transform(imagelabels)
y = to_categorical(y,4)
print(X.shape)
print(y.shape)

(199, 150, 150, 3)
(199, 4)


In [6]:
X_train,X_valid,y_train,y_valid = train_test_split(X,y,test_size=0.2,random_state=42)
X_train,X_test,y_train,y_test = train_test_split(X_train,y_train,test_size=0.2,random_state=42)
X_train.shape,y_train.shape,X_valid.shape,y_valid.shape,X_test.shape,y_test.shape

((127, 150, 150, 3),
 (127, 4),
 (40, 150, 150, 3),
 (40, 4),
 (32, 150, 150, 3),
 (32, 4))

In [7]:
batch_size = 64
epochs = 100
num_classes = y.shape[1]

In [8]:
from tensorflow.keras.applications import ResNet152V2
#from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras import Model, layers
pre_trained_model = ResNet152V2(input_shape = (150, 150, 3),include_top=False, weights='imagenet')
for layer in pre_trained_model.layers:
  layer.trainable = False
  
pre_trained_model.summary()

last_layer = pre_trained_model.get_layer('conv5_block2_out')
print('last layer output shape: ', last_layer.output_shape)
last_output = last_layer.output

234553344/234545216 [==============================] - 2s 0us/step
Model: "resnet152v2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 150, 150, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 156, 156, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 75, 75, 64)   9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
pool1_pad (ZeroPadding2D)       (None, 77, 77, 64)   0           conv1_conv[0][0]                 
_____________________

In [15]:
# Flatten the output layer to 1 dimension
x = layers.Flatten()(last_output)
# Add a fully connected layer with 1,024 hidden units and ReLU activation
x = layers.Dense(1024, activation='relu')(x)
x = layers.Dense(512, activation='relu')(x)
# Add a dropout rate of 0.3
x = layers.Dropout(0.2)(x)                  
# Add a final sigmoid layer for classification
x = layers.Dense  (4, activation='softmax')(x)           

model = Model( pre_trained_model.input, x) 
model.summary()

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 150, 150, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 156, 156, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 75, 75, 64)   9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
pool1_pad (ZeroPadding2D)       (None, 77, 77, 64)   0           conv1_conv[0][0]                 
_______________________________________________________________________________________

In [16]:
from tensorflow.keras.optimizers import Adam

from tensorflow.keras.optimizers import RMSprop
model.compile(loss='categorical_crossentropy',
              optimizer=Adam(lr=0.001),
              metrics=['accuracy'])
print('Compiled!')

Compiled!


In [17]:
imagegen = ImageDataGenerator(featurewise_center=False,
                             samplewise_center=False,
                             featurewise_std_normalization=False,
                             samplewise_std_normalization=False,
                            rotation_range=60,
                              zoom_range=0.1,
                              width_shift_range=0.1,
                              height_shift_range=0.1,
                              shear_range=0.1,
                              fill_mode='reflect')
imagegen.fit(X_train)


In [19]:
from tensorflow.keras.callbacks import ModelCheckpoint,ReduceLROnPlateau
checkpoint = ModelCheckpoint(filepath = 'cnn3.hdf5',monitor='val_accuracy' ,verbose = 1, save_best_only = True)
model_dropout = model.fit_generator(imagegen.flow(X_train,y_train, batch_size=64),epochs=100,verbose=2,
                          validation_data=(X_valid, y_valid),steps_per_epoch=X_train.shape[0] //64,callbacks=[checkpoint],shuffle=True)

Epoch 1/100

Epoch 00001: val_accuracy improved from -inf to 0.47500, saving model to cnn3.hdf5
1/1 - 11s - loss: 100.4612 - accuracy: 0.4219 - val_loss: 61.5860 - val_accuracy: 0.4750
Epoch 2/100

Epoch 00002: val_accuracy improved from 0.47500 to 0.77500, saving model to cnn3.hdf5
1/1 - 11s - loss: 59.7009 - accuracy: 0.4762 - val_loss: 2.4835 - val_accuracy: 0.7750
Epoch 3/100

Epoch 00003: val_accuracy did not improve from 0.77500
1/1 - 7s - loss: 4.4858 - accuracy: 0.6406 - val_loss: 39.4128 - val_accuracy: 0.5250
Epoch 4/100

Epoch 00004: val_accuracy did not improve from 0.77500
1/1 - 7s - loss: 40.1893 - accuracy: 0.5397 - val_loss: 38.5807 - val_accuracy: 0.5250
Epoch 5/100

Epoch 00005: val_accuracy did not improve from 0.77500
1/1 - 7s - loss: 45.6894 - accuracy: 0.4219 - val_loss: 10.9944 - val_accuracy: 0.5250
Epoch 6/100

Epoch 00006: val_accuracy did not improve from 0.77500
1/1 - 7s - loss: 14.7313 - accuracy: 0.4921 - val_loss: 14.3632 - val_accuracy: 0.4750
Epoch 7/10

In [20]:
imagedetails1 = []
imagelabels1 = []
IMG_SIZE1=150
gallina_dir='/content/drive/My Drive/pp_used/gallina'
cane_dir = '/content/drive/My Drive/pp_used/cane'
mucca_dir = '/content/drive/My Drive/pp_used/mucca'
pecora_dir = '/content/drive/My Drive/pp_used/pecora'

In [21]:
def read_image1(label,DIR):
    
    for img in tqdm(os.listdir(DIR)):
    #for img in DIR:
        path = os.path.join(DIR,img)
        #checking image type
        _, ftype = os.path.splitext(path)
        if ftype == ".jpeg":
       
            image = cv2.imread(path,cv2.IMREAD_COLOR)
            image = cv2.resize(image, (IMG_SIZE1,IMG_SIZE1),interpolation = cv2.INTER_AREA)
            imagedetails1.append(np.array(image))
            imagelabels1.append(str(label))

read_image1("gallina",gallina_dir)
read_image1("cane",cane_dir)
read_image1("mucca",mucca_dir)
read_image1("pecora",pecora_dir)

100%|██████████| 990/990 [02:20<00:00,  7.06it/s]


In [22]:
encoder = LabelEncoder()
X1 = np.array(imagedetails1)
X1 = X1/255
y1 = encoder.fit_transform(imagelabels1)
y1 = to_categorical(y1,4)
print(X1.shape)
print(y1.shape)

(3646, 150, 150, 3)
(3646, 4)


In [23]:
X_train1,X_valid1,y_train1,y_valid1 = train_test_split(X1,y1,test_size=0.2,random_state=42)
X_train1,X_test1,y_train1,y_test1 = train_test_split(X_train1,y_train1,test_size=0.2,random_state=42)
X_train1.shape,y_train1.shape,X_valid1.shape,y_valid1.shape,X_test1.shape,y_test1.shape

((2332, 150, 150, 3),
 (2332, 4),
 (730, 150, 150, 3),
 (730, 4),
 (584, 150, 150, 3),
 (584, 4))

In [24]:
batch_size = 64
epochs = 100
num_classes = y1.shape[1]

In [26]:
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras import Model, layers
pre_trained_model1 = InceptionV3(input_shape = (150, 150, 3),include_top=False, weights='imagenet')
for layer in pre_trained_model1.layers:
  layer.trainable = False
  
from keras import Model
layer_name = 'mixed7'
my_model = Model(inputs=pre_trained_model1.input,outputs=pre_trained_model1.get_layer(layer_name).output)
my_model.summary()

Model: "functional_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, 150, 150, 3) 0                                            
__________________________________________________________________________________________________
conv2d_94 (Conv2D)              (None, 74, 74, 32)   864         input_3[0][0]                    
__________________________________________________________________________________________________
batch_normalization_94 (BatchNo (None, 74, 74, 32)   96          conv2d_94[0][0]                  
__________________________________________________________________________________________________
activation_94 (Activation)      (None, 74, 74, 32)   0           batch_normalization_94[0][0]     
_______________________________________________________________________________________

In [27]:
from keras import models         
model1 = models.Sequential()
model1.add(my_model)
from keras.layers import Dense, Conv2D, MaxPooling2D, BatchNormalization, GlobalAveragePooling2D
model1.add(Conv2D(128,(3,3),activation='relu',padding='same'))
model1.add(MaxPooling2D((2,2),padding='same'))
model1.add(Conv2D(256,(3,3),activation='relu',padding='same'))
model1.add(MaxPooling2D((2,2),padding='same'))
model1.add(GlobalAveragePooling2D())
model1.add(Dense(64,activation='relu'))
model1.add(BatchNormalization())
model1.add(Dense(4,activation='softmax'))
model1.layers[7].trainable = False

In [28]:
from tensorflow.keras.optimizers import Adam

from tensorflow.keras.optimizers import RMSprop
model1.compile(loss='categorical_crossentropy',
              optimizer=Adam(lr=0.001),
              metrics=['accuracy'])
print('Compiled!')

Compiled!


In [29]:
imagegen1 = ImageDataGenerator(featurewise_center=False,
                             samplewise_center=False,
                             featurewise_std_normalization=False,
                             samplewise_std_normalization=False,
                            rotation_range=60,
                              zoom_range=0.1,
                              width_shift_range=0.1,
                              height_shift_range=0.1,
                              shear_range=0.1,
                              fill_mode='reflect')
imagegen.fit(X_train)

In [30]:
from tensorflow.keras.callbacks import ModelCheckpoint,ReduceLROnPlateau

checkpoint = ModelCheckpoint(filepath = 'cnn3.hdf5',monitor='val_accuracy' ,verbose = 1, save_best_only = True)
reduce_lr=ReduceLROnPlateau(monitor='val_loss',factor=0.1,patience=2,min_lr=0.00001,mode='auto')
model_dropout1 = model1.fit_generator(imagegen1.flow(X_train,y_train, batch_size=32),epochs=100,callbacks=[checkpoint,reduce_lr],verbose=2,
                          validation_data=imagegen.flow(X_valid, y_valid,batch_size=32),shuffle=True,steps_per_epoch=X_train.shape[0] // 32)

Epoch 1/100

Epoch 00001: val_accuracy improved from -inf to 0.47500, saving model to cnn3.hdf5
3/3 - 6s - loss: 1.1538 - accuracy: 0.3789 - val_loss: 0.8968 - val_accuracy: 0.4750
Epoch 2/100

Epoch 00002: val_accuracy improved from 0.47500 to 0.67500, saving model to cnn3.hdf5
3/3 - 5s - loss: 0.8033 - accuracy: 0.4737 - val_loss: 0.6394 - val_accuracy: 0.6750
Epoch 3/100

Epoch 00003: val_accuracy did not improve from 0.67500
3/3 - 4s - loss: 0.4304 - accuracy: 0.8105 - val_loss: 0.6582 - val_accuracy: 0.5750
Epoch 4/100

Epoch 00004: val_accuracy improved from 0.67500 to 0.75000, saving model to cnn3.hdf5
3/3 - 5s - loss: 0.4414 - accuracy: 0.7812 - val_loss: 0.5710 - val_accuracy: 0.7500
Epoch 5/100

Epoch 00005: val_accuracy improved from 0.75000 to 0.80000, saving model to cnn3.hdf5
3/3 - 5s - loss: 0.3226 - accuracy: 0.8947 - val_loss: 0.6889 - val_accuracy: 0.8000
Epoch 6/100

Epoch 00006: val_accuracy did not improve from 0.80000
3/3 - 4s - loss: 0.3102 - accuracy: 0.8958 - v

In [32]:
test_eval = model1.evaluate(X_test1, y_test1, verbose=2)

19/19 - 19s - loss: 5.6109 - accuracy: 0.4521


In [33]:
y_pred = model.predict(X_test)
y_pred = np.argmax(y_pred,axis=1).astype(int)
y_true = np.argmax(y_test,axis = 1).astype(int)

In [147]:
imagedetails2 = []
imagelabels2 = []
IMG_SIZE1=150
gallina_dir='/content/drive/My Drive/test_data'

def read_image2(label,DIR):
    
    for img in tqdm(os.listdir(DIR)):
    #for img in DIR:
        path = os.path.join(DIR,img)
        #checking image type
        _, ftype = os.path.splitext(path)
        image = cv2.imread(path,cv2.IMREAD_COLOR)
        image = cv2.resize(image, (IMG_SIZE1,IMG_SIZE1),interpolation = cv2.INTER_AREA)
        imagedetails2.append(np.array(image))
        imagelabels2.append(str(label))

read_image2("gallina",gallina_dir)
print(imagedetails2)
print(imagelabels2)

100%|██████████| 1/1 [00:00<00:00, 145.07it/s]

[array([[[ 22,  28,  23],
        [ 25,  30,  25],
        [ 29,  33,  28],
        ...,
        [ 42,  48,  37],
        [ 44,  50,  39],
        [ 49,  55,  44]],

       [[ 30,  36,  31],
        [ 31,  36,  31],
        [ 32,  36,  31],
        ...,
        [ 44,  48,  37],
        [ 46,  50,  39],
        [ 51,  55,  44]],

       [[ 37,  43,  38],
        [ 37,  42,  37],
        [ 36,  40,  35],
        ...,
        [ 45,  47,  37],
        [ 47,  49,  39],
        [ 52,  54,  44]],

       ...,

       [[ 93,  96, 101],
        [ 77,  80,  85],
        [ 65,  68,  73],
        ...,
        [228, 232, 233],
        [229, 233, 234],
        [229, 233, 234]],

       [[ 71,  74,  78],
        [ 62,  64,  68],
        [ 56,  57,  62],
        ...,
        [223, 227, 228],
        [224, 228, 229],
        [224, 228, 229]],

       [[ 60,  61,  65],
        [ 61,  62,  66],
        [ 56,  57,  61],
        ...,
        [221, 225, 226],
        [222, 226, 227],
        [222, 226, 227]

In [148]:
encoder = LabelEncoder()
X2 = np.array(imagedetails2)
X2 = X2/255
y2 = encoder.fit_transform(imagelabels2)
y2 = to_categorical(y2,4)
print(y2)

[[1. 0. 0. 0.]]


In [150]:
test_eval = model1.evaluate(X2, y2, verbose=2)
y_pred = model1.predict(X2)
print(y_pred)
y_pred = np.argmax(y_pred,axis=1).astype(int)
print(y_pred)
y_true = np.argmax(y2,axis = 1).astype(int)
print(y_true)

1/1 - 0s - loss: 0.0148 - accuracy: 1.0000
[[9.8528463e-01 1.4714730e-02 6.4130495e-07 5.7371931e-08]]
[0]
[0]


In [158]:
if(y_pred==0):
  imagelabels2[0] = "diseased"
  encoder = LabelEncoder()
  y2 = encoder.fit_transform(imagelabels2)
  y2 = to_categorical(y2,4)
  y_pred = model.predict(X2)
  y_pred = np.argmax(y_pred,axis=1).astype(int)
  if(y_pred==0):
    print("It's a diseased chicken")
  else:
    print("It's a healthy chicken")
  


It's a diseased chicken
